# Notebook for looking at recorded phone usage


In [2]:
import os
import sys
import glob
import pandas as pd

import tzlocal
from zoneinfo import ZoneInfo
from datetime import timedelta

sys.path.insert(0, "../")
from db import HindsightDB
import utils

In [3]:
db = HindsightDB()

In [16]:
frames_df = db.get_frames()
frames_df = utils.add_datetimes(frames_df)
frames_df = frames_df.sort_values(by='datetime_local')

In [17]:
set(frames_df['application'])

{'android',
 'co-hinge-app',
 'com-americanexpress-android-acctsvcs-us',
 'com-android-chrome',
 'com-android-intentresolver',
 'com-android-phone',
 'com-android-pixeldisplayservice',
 'com-android-settings',
 'com-android-systemui',
 'com-android-vending',
 'com-chase-sig-android',
 'com-connor-hindsight',
 'com-discoverfinancial-mobile',
 'com-facebook-katana',
 'com-fidelity-android',
 'com-github-android',
 'com-google-android-GoogleCamera',
 'com-google-android-apps-chromecast-app',
 'com-google-android-apps-docs-editors-docs',
 'com-google-android-apps-maps',
 'com-google-android-apps-messaging',
 'com-google-android-apps-nexuslauncher',
 'com-google-android-apps-photos',
 'com-google-android-apps-recorder',
 'com-google-android-apps-tachyon',
 'com-google-android-apps-wellbeing',
 'com-google-android-as',
 'com-google-android-calculator',
 'com-google-android-contacts',
 'com-google-android-deskclock',
 'com-google-android-dialer',
 'com-google-android-ext-services',
 'com-goog

In [18]:
frames_df = frames_df.loc[frames_df['application'] != 'com-android-systemui'] # Lines up better with phone

In [19]:
frames_df['time_difference'] = frames_df['datetime_local'].diff()

In [20]:
new_usage_threshold = timedelta(seconds=120)
new_usage_start = (frames_df['time_difference'] > new_usage_threshold)
frames_df['usage_id'] = new_usage_start.cumsum()

In [21]:
frames_df['datetime_to_day'] = frames_df['datetime_local'].dt.floor('D')

In [23]:
frames_df.groupby(['datetime_to_day']).usage_id.nunique()

datetime_to_day
2024-05-14 00:00:00-04:00    10
2024-05-15 00:00:00-04:00    13
2024-05-16 00:00:00-04:00    50
2024-05-17 00:00:00-04:00    50
2024-05-18 00:00:00-04:00    64
2024-05-19 00:00:00-04:00    41
2024-05-20 00:00:00-04:00    60
2024-05-21 00:00:00-04:00    51
2024-05-22 00:00:00-04:00    45
2024-05-23 00:00:00-04:00    41
2024-05-24 00:00:00-04:00    14
2024-05-25 00:00:00-04:00    22
2024-05-26 00:00:00-04:00     6
2024-05-27 00:00:00-04:00     6
2024-05-28 00:00:00-04:00    31
2024-05-29 00:00:00-04:00    18
2024-05-30 00:00:00-04:00    31
2024-05-31 00:00:00-04:00    32
2024-06-01 00:00:00-04:00    28
2024-06-02 00:00:00-04:00    39
2024-06-03 00:00:00-04:00    70
2024-06-04 00:00:00-04:00    75
2024-06-05 00:00:00-04:00    44
Name: usage_id, dtype: int64

In [24]:
usage_durations = frames_df.groupby('usage_id')['datetime_local'].agg([min, max])
usage_durations['duration'] = usage_durations['max'] - usage_durations['min']
usage_durations['day'] = usage_durations['min'].dt.floor('D')

/var/folders/c_/2c9vmfhd35bgwc_6h0q80d180000gn/T/ipykernel_33218/410602690.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  usage_durations = frames_df.groupby('usage_id')['datetime_local'].agg([min, max])
/var/folders/c_/2c9vmfhd35bgwc_6h0q80d180000gn/T/ipykernel_33218/410602690.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  usage_durations = frames_df.groupby('usage_id')['datetime_local'].agg([min, max])


In [25]:
usage_durations.groupby(['day']).duration.sum()

day
2024-05-14 00:00:00-04:00   0 days 00:07:51.997000219
2024-05-15 00:00:00-04:00   0 days 00:37:17.220000507
2024-05-16 00:00:00-04:00   0 days 01:55:45.473000772
2024-05-17 00:00:00-04:00   0 days 01:49:18.412001131
2024-05-18 00:00:00-04:00   0 days 01:41:29.695998193
2024-05-19 00:00:00-04:00   0 days 01:16:05.827998876
2024-05-20 00:00:00-04:00   0 days 02:23:03.849999902
2024-05-21 00:00:00-04:00   0 days 01:29:33.749000064
2024-05-22 00:00:00-04:00   0 days 01:35:20.889999869
2024-05-23 00:00:00-04:00   0 days 03:22:15.644998547
2024-05-24 00:00:00-04:00   0 days 00:13:07.250000003
2024-05-25 00:00:00-04:00   0 days 00:53:03.009000059
2024-05-26 00:00:00-04:00   0 days 00:19:09.561999559
2024-05-27 00:00:00-04:00   0 days 00:14:35.391000031
2024-05-28 00:00:00-04:00   0 days 01:55:30.561999322
2024-05-29 00:00:00-04:00   0 days 00:15:30.025000332
2024-05-30 00:00:00-04:00   0 days 00:48:49.154999974
2024-05-31 00:00:00-04:00   0 days 00:46:40.722999808
2024-06-01 00:00:00-04:0

In [26]:
usage_durations.duration.sum()

Timedelta('1 days 07:39:21.341998096')